In [30]:
# get kaggle json to download the dataset 
! pip install kaggle
! rm -rf ~/.kaggle
! mkdir ~/.kaggle
! wget https://transfer.sh/Bh4pIY/kaggle.json
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

--2022-04-01 23:37:05--  https://transfer.sh/Bh4pIY/kaggle.json
Resolving transfer.sh (transfer.sh)... 144.76.136.153, 2a01:4f8:200:1097::2
Connecting to transfer.sh (transfer.sh)|144.76.136.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71 [application/json]
Saving to: ‘kaggle.json.2’

kaggle.json.2       100%[===================>]      71  --.-KB/s    in 0s      

2022-04-01 23:37:06 (505 KB/s) - ‘kaggle.json.2’ saved [71/71]



In [ ]:
# torchvison inbuilt dataset is not working, hence have to use kaggle   
! kaggle datasets download jessicali9530/celeba-dataset

 99% 1.31G/1.33G [00:12<00:00, 97.0MB/s]
100% 1.33G/1.33G [00:13<00:00, 110MB/s] 


In [ ]:
# unzip the dataset
!unzip celeba-dataset.zip

In [ ]:
# clone and install all the required repos and packages
! git clone https://github.com/skp-github/MaskTheFace.git
! git clone https://github.com/skp-github/3DDFA.git
! git clone https://github.com/skp-github/3DDFA_V2.git
! pip install mtcnn
! pip install mediapipe
! pip install dotmap
! pip install face-alignment

### Installation will take time **(10-20 mins approx)**


In [ ]:
# setup and install all repos 

# 1. setup mask the face repo 
%cd MaskTheFace
! pip install -r requirements.txt
%cd ..

# 2. Setup 3DDFA repo
%cd 3DDFA/
! pip install -r requirements.txt
%cd utils/cython/
! python setup.py build_ext -i
%cd ../../..

# 3. Setup 3DDFA_V2
%cd 3DDFA_V2
!sh ./build.sh
%cd ..

## Create subset dataset from 200k images


---



---



In [ ]:
import torchvision
from torchvision.transforms import transforms
import cv2 
import matplotlib.pyplot as plt
import random 
import pandas as pd


In [ ]:
!rm -rf '/content/sample_dataset/'
!mkdir '/content/sample_dataset/'

In [ ]:
import glob
import shutil
import os
# select random sample of images 
# you can change sample size(number of images for the experiment) to increase the dataset 
def create_subset_dataset(img_dataset_path,sample_size=150):
  # files number list i.e 000001.py
  files_list = []
  # image absolute path i.e 
  dir_path = []
  while sample_size !=0: 
    # generate a random image number
    image_number = str(random.randint(1, 10000))
    # pad the random number i.e 23 => 0000 (total 6 digits required)
    trailing_zeros = "0"*(6 - len(image_number) )
    # image_number = "0000"+"23"+"+".jpg" => 000023.jpg
    image_number = trailing_zeros + image_number + ".jpg"
    # check if this image is already used 
    if image_number in files_list:
      continue
    #create image path i.e "/content/img_align_celeba/img_align_celeba/"+"000023.jpg" => "/content/img_align_celeba/img_align_celeba/000023.jpg"
    image_path = img_dataset_path + image_number 
    # append the image_number i.e "000023.jpg"
    files_list.append(image_number)
    # append the image_path i.e "/content/img_align_celeba/img_align_celeba/000023.jpg"
    dir_path.append(image_path)
    # reduce the sample size
    sample_size -= 1
  # copy images from dataset folder to sample_dataset folder
  base_dst_dir = '/content/sample_dataset/'
  for i  in range(0, len(files_list)):
    dst_dir = base_dst_dir + files_list[i]
    shutil.copy(dir_path[i], dst_dir)

create_subset_dataset('/content/img_align_celeba/img_align_celeba/')
# remove 1GB dataset 
!rm -rf img_align_celeba/


['/content/img_align_celeba/img_align_celeba/005377.jpg', '/content/img_align_celeba/img_align_celeba/001673.jpg', '/content/img_align_celeba/img_align_celeba/009264.jpg', '/content/img_align_celeba/img_align_celeba/004107.jpg', '/content/img_align_celeba/img_align_celeba/007770.jpg', '/content/img_align_celeba/img_align_celeba/002576.jpg', '/content/img_align_celeba/img_align_celeba/000475.jpg', '/content/img_align_celeba/img_align_celeba/000389.jpg', '/content/img_align_celeba/img_align_celeba/007445.jpg', '/content/img_align_celeba/img_align_celeba/002136.jpg', '/content/img_align_celeba/img_align_celeba/002947.jpg', '/content/img_align_celeba/img_align_celeba/009364.jpg', '/content/img_align_celeba/img_align_celeba/009431.jpg', '/content/img_align_celeba/img_align_celeba/004634.jpg', '/content/img_align_celeba/img_align_celeba/006099.jpg', '/content/img_align_celeba/img_align_celeba/000549.jpg', '/content/img_align_celeba/img_align_celeba/000400.jpg', '/content/img_align_celeba/img

# Convert images from the subset to masked images

---



---



In [ ]:
# mask the sample images
%cd MaskTheFace/
! python mask_the_face.py --path '/content/sample_dataset/'
%cd ..

# 3DDFA on masked dataset

---



---



In [ ]:
# landmarks from 3DDFA (68 points) => shape = (68,3)

# remove all files from 3ddfa processed folder if any 
!rm -rf '/content/3ddfa_images/'
# create 3ddfa processed folder if not already there  
!mkdir '/content/3ddfa_images/'
# remove files from 3ddfa processed folder sub folder landmarks  if any
!rm -rf '/content/3ddfa_images/landmarks/'
# create 3ddfa processed folder sub folder to store landmarks  if not already there
!mkdir '/content/3ddfa_images/landmarks/'
# cd in the repo
%cd 3DDFA/
# run main.py over the masked images , 3DDFA (images and landmarks are stored in the folders mentioned above )
!python main.py -f '/content/masked_images/'
%cd ..

# 3DDFA_V2 on masked dataset

---



---



In [ ]:
# 3DDFA2 landmarks points 

# exactly like 3DDFA similar process with 3DDFA_V2
!rm -rf '/content/3ddfa2_images/'
!mkdir '/content/3ddfa2_images/'
!rm -rf '/content/3ddfa2_images/landmarks/'
!mkdir '/content/3ddfa2_images/landmarks/'
%cd 3DDFA_V2/

/content/3DDFA_V2


In [ ]:
# import packages and files  
import cv2
import yaml

from FaceBoxes import FaceBoxes
from TDDFA import TDDFA
from utils.render import render
from utils.depth import depth
from utils.pncc import pncc
from utils.uv import uv_tex
from utils.pose import viz_pose
from utils.serialization import ser_to_ply, ser_to_obj
from utils.functions import draw_landmarks, get_suffix

import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm
import os
import numpy as np


In [ ]:
# load config
cfg = yaml.load(open('configs/mb1_120x120.yml'), Loader=yaml.SafeLoader)
face_boxes = FaceBoxes()
tddfa = TDDFA(gpu_mode=False, **cfg)
img_path = '/content/masked_images/'
dense_flag = False
path, dirs, files = os.walk(img_path).__next__()
destination_folder = "/content/3ddfa2_images/"
for f in tqdm(files):
  img_fp = path+f
  img_ori = cv2.imread(img_fp)
  boxes = face_boxes(img_ori)
  param_lst, roi_box_lst = tddfa(img_ori, boxes)
  ver_lst = tddfa.recon_vers(param_lst, roi_box_lst, dense_flag=dense_flag)
  wfp = destination_folder +"landmarks/" +'{}.txt'.format(img_fp.split("/")[3][:-4])
  # if landmarks generated
  if ver_lst:
    # save landmarks 
    np.savetxt(wfp, ver_lst[0], fmt='%.3f')
    viz_wfp = destination_folder +'{}'.format(img_fp.split("/")[3]) 
    # save images with landmarks  
    draw_landmarks(img_ori, ver_lst, wfp=viz_wfp,dense_flag=dense_flag)
%cd ..


# Face-Alignment(Arianb) on masked dataset

---



---



In [ ]:
# face-alignment landmarks points 
!rm -rf '/content/face_alignment_images/'
!mkdir '/content/face_alignment_images/'
!rm -rf '/content/face_alignment_images/landmarks/'
!mkdir '/content/face_alignment_images/landmarks/'

In [ ]:
# import required packages 
import face_alignment
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

def draw_landmarks(img, pts, style='fancy', wfp=None, show_flag=True, **kwargs):
    """Draw landmarks using matplotlib"""
    height, width = img.shape[:2]
    plt.figure(figsize=(12, height / width * 12))
    plt.imshow(img[..., ::-1])
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.axis('off')

    dense_flag = kwargs.get('dense_flag')

    if not type(pts) in [tuple, list]:
        pts = [pts]
    for i in range(len(pts)):
        if dense_flag:
            plt.plot(pts[i][0, ::6], pts[i][1, ::6], 'o', markersize=0.4, color='c', alpha=0.7)
        else:
            alpha = 0.8
            markersize = 4
            lw = 1.5
            color = kwargs.get('color', 'w')
            markeredgecolor = kwargs.get('markeredgecolor', 'black')

            nums = [0, 17, 22, 27, 31, 36, 42, 48, 60, 68]

            # close eyes and mouths
            plot_close = lambda i1, i2: plt.plot([pts[i][0, i1], pts[i][0, i2]], [pts[i][1, i1], pts[i][1, i2]],
                                                 color=color, lw=lw, alpha=alpha - 0.1)
            plot_close(41, 36)
            plot_close(47, 42)
            plot_close(59, 48)
            plot_close(67, 60)

            for ind in range(len(nums) - 1):
                l, r = nums[ind], nums[ind + 1]
                plt.plot(pts[i][0, l:r], pts[i][1, l:r], color=color, lw=lw, alpha=alpha - 0.1)

                plt.plot(pts[i][0, l:r], pts[i][1, l:r], marker='o', linestyle='None', markersize=markersize,
                         color=color,
                         markeredgecolor=markeredgecolor, alpha=alpha)
            
    if wfp is not None:
        plt.savefig(wfp, dpi=150)
        print(f'Save visualization result to {wfp}')

    if show_flag:
        plt.show()


fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False, device="cpu")
img_path = '/content/masked_images/'
path, dirs, files = os.walk(img_path).__next__()
destination_folder = '/content/face_alignment_images/'
for f in tqdm(files):
  img_fp = path+f
  img_ori = cv2.imread(img_fp)
  preds = fa.get_landmarks(img_ori)
  wfp = destination_folder +"landmarks/" +'{}.txt'.format(img_fp.split("/")[3][:-4])
  # if landmarks generated
  if preds:
    #save landmarks 
    np.savetxt(wfp, np.swapaxes(preds[0],0,1), fmt='%.3f')
    viz_wfp = destination_folder +'{}'.format(img_fp.split("/")[3]) 
    # save image with 
    draw_landmarks(img_ori, wfp=viz_wfp, pts=np.swapaxes(preds[0],0,1))

# **Side Note**
#### ***sample_dataset*** = certain number of random images picked for experiment(could have been avioded)
#### ***masked_images*** = folder with masked images (uses sampled dataset directory)
#### ***3ddfa_images*** (all 3ddfa processed images) = have a subfolder with landmarks  i.e 000001.py (image names) i.e 000001.txt (landmarks , (68,3))
#### ***3ddfa2_images*** (all 3ddfa_v2 processed images) = have a subfolder with landmarks  i.e 000001.py (image names) i.e 000001.txt (landmarks, (68,3))
#### ***face_alignment_images*** (all face_alignment_images images) = have a subfolder with landmarks i.e 000001.py (image names) i.e 000001.txt (landmarks, (68,3)) 

# **Ground truth landmarks are in list_landmarks_align_celeba.csv**
#### for more details check this https://www.kaggle.com/datasets/jessicali9530/celeba-dataset?select=list_landmarks_align_celeba.csv

